## The Runnable and Runnable Sequence Classes

In [2]:
%load_ext dotenv
%dotenv

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [4]:
chat_template = ChatPromptTemplate.from_messages([('human', "I've recently adopted a {pet}, which is a {breed}. Could you suggest some training tips?? \n")])

In [5]:
chat = ChatOpenAI(model='gpt-4o-mini', seed=365, temperature=0, max_tokens=100)

In [6]:
chain = chat_template | chat

In [7]:
RunnablePassthrough().invoke([1,2,3])

[1, 2, 3]

In [8]:
chat_template_tools = ChatPromptTemplate.from_template("""
What are the five most important tools a {job_title} needs?
Answer only by listing the tools.
""")

chat_template_strategy = ChatPromptTemplate.from_template("""
Considering the tools provided, develop a strategy for effectively learning and mastering them:
{tools}
""")

In [9]:
chat_template_tools

ChatPromptTemplate(input_variables=['job_title'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['job_title'], input_types={}, partial_variables={}, template='\nWhat are the five most important tools a {job_title} needs?\nAnswer only by listing the tools.\n'), additional_kwargs={})])

In [10]:
chat_template_strategy

ChatPromptTemplate(input_variables=['tools'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tools'], input_types={}, partial_variables={}, template='\nConsidering the tools provided, develop a strategy for effectively learning and mastering them:\n{tools}\n'), additional_kwargs={})])

In [11]:
string_parser = StrOutputParser()

In [12]:
chain_tools = chat_template_tools | chat | string_parser | {'tools': RunnablePassthrough()}
chain_strategy = chat_template_strategy | chat | string_parser

In [13]:
print(chain_tools.invoke({'job_title': 'data scientist'}))

{'tools': '1. Python\n2. R\n3. SQL\n4. Jupyter Notebook\n5. Git'}


In [14]:
print(chain_strategy.invoke({'tools': """
1. Python
2. R
3. SQL
4. Jupyter Notebook
5. TensorFlow/PyTorch
"""}))

To effectively learn and master Python, R, SQL, Jupyter Notebook, and TensorFlow/PyTorch, you can follow a structured strategy that incorporates foundational knowledge, practical application, and continuous improvement. Here’s a step-by-step approach:

### Step 1: Establish a Learning Foundation

#### 1. **Python**
   - **Resources**: Use online platforms like Codecademy, Coursera, or freeCodeCamp to learn Python basics.
   - **Practice**: Solve problems on


In [15]:
chain_combined = chain_tools | chain_strategy

In [16]:
print(chain_combined.invoke({'job_title': 'data scientist'}))

To effectively learn and master Python, R, SQL, Jupyter Notebook, and Git, you can follow a structured strategy that incorporates a mix of theoretical understanding, practical application, and project-based learning. Here’s a step-by-step approach:

### 1. Set Clear Goals
- **Define Objectives**: Determine what you want to achieve with each tool. For example, do you want to use Python for data analysis, R for statistical modeling, SQL for database management, Jupyter Notebook for interactive


## Graphing Runnables

In [17]:
chat_template_tools = ChatPromptTemplate.from_template('''
What are the five most important tools a {job title} needs?
Answer only by listing the tools.
''')

chat_template_strategy = ChatPromptTemplate.from_template('''
Considering the tools provided, develop a strategy for effectively learning and mastering them:
{tools}
''')

In [18]:
chat = ChatOpenAI(model_name = 'gpt-4', 
                  seed=365,
                  temperature = 0,
                  max_tokens = 100)

In [19]:
string_parser = StrOutputParser()

In [20]:
chain_long = (chat_template_tools | chat | string_parser | {'tools':RunnablePassthrough()} | 
              chat_template_strategy | chat | string_parser)

In [21]:
chain_long.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
     +-------------+       
     | Passthrough |       
     +-------------+       
            *              
            *              
            *       

## Runnable Parallel

In [22]:
from langchain_core.runnables import RunnableParallel

In [23]:
chat_template_books = ChatPromptTemplate.from_template('''
Suggest three of the best intermediate-level {programming language} books. 
Answer only by listing the books.
''')

chat_template_projects = ChatPromptTemplate.from_template('''
Suggest three interesting {programming language} projects suitable for intermediate-level programmers. 
Answer only by listing the projects.
''')

In [24]:
string_parser = StrOutputParser()

In [25]:
chain_books = chat_template_books | chat | string_parser
chain_projects = chat_template_projects | chat | string_parser

In [26]:
chain_parallel = RunnableParallel({'books': chain_books, 'projects': chain_projects})

In [27]:
%%time
chain_parallel.invoke({'programming language': 'Python'})

CPU times: total: 0 ns
Wall time: 4.66 s


{'books': '1. "Fluent Python: Clear, Concise, and Effective Programming" by Luciano Ramalho\n2. "Python Cookbook: Recipes for Mastering Python 3" by David Beazley and Brian K. Jones\n3. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin',
 'projects': '1. Building a Weather Forecasting Application using APIs.\n2. Developing a Personal Finance Tracker with Data Visualization.\n3. Creating a Text-based Adventure Game with multiple scenarios and outcomes.'}

In [28]:
chain_parallel.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            

In [29]:
%%time
chain_books.invoke({'programming language': 'Python'})

CPU times: total: 0 ns
Wall time: 3.06 s


'1. "Fluent Python: Clear, Concise, and Effective Programming" by Luciano Ramalho\n2. "Python Cookbook: Recipes for Mastering Python 3" by David Beazley and Brian K. Jones\n3. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin'

In [30]:
%%time
chain_projects.invoke({'programming language': 'Python'})

CPU times: total: 0 ns
Wall time: 1.64 s


'1. Building a Web Scraper using BeautifulSoup\n2. Developing a simple Machine Learning model using Scikit-learn\n3. Creating a GUI application using Tkinter'

## Piping Runnable Parallel with another Runnable

In [31]:
chat = ChatOpenAI(model='gpt-4o-mini', seed=365, temperature=0, max_tokens=500)

In [32]:
chat_template_books = ChatPromptTemplate.from_template('''
Suggest three of the best intermediate-level {programming language} books. 
Answer only by listing the books.
''')

chat_template_projects = ChatPromptTemplate.from_template('''
Suggest three interesting {programming language} projects suitable for intermediate-level programmers. 
Answer only by listing the projects.
''')

chat_template_time = ChatPromptTemplate.from_template("""
I'm an intermediate level programmer.
     
Consider the following literature:
{books}

Also, consider the following projects:
{projects}

Roughly how much time would it take me to complete the literature and the projects?
""")

In [33]:
chain_books = chat_template_books | chat | string_parser
chain_projects = chat_template_projects | chat | string_parser

In [34]:
chain_parallel = RunnableParallel({'books': chain_books, 'projects': chain_projects})

In [35]:
chain_parallel.invoke({'programming language': 'Python'})

{'books': '1. "Fluent Python" by Luciano Ramalho  \n2. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin  \n3. "Python Cookbook" by David Beazley and Brian K. Jones  ',
 'projects': '1. Personal Finance Tracker with Data Visualization  \n2. Web Scraper with Automated Reporting  \n3. Chatbot using Natural Language Processing (NLP)'}

In [36]:
chain_parallel.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            

In [37]:
chain_time1 = RunnableParallel({'books': chain_books, 'projects': chain_projects}) | chat_template_time | chat | string_parser

In [38]:
chain_time2 ={'books': chain_books, 'projects': chain_projects} | chat_template_time | chat | string_parser

In [39]:
print(chain_time2.invoke({'programming language': 'Python'}))

The time it takes to complete the literature and projects can vary based on your current skill level, the depth of understanding you wish to achieve, and the complexity of the projects. However, I can provide a rough estimate for each.

### Literature

1. **"Fluent Python" by Luciano Ramalho**  
   - **Estimated Time**: 4-6 weeks  
   - This book is comprehensive and covers advanced Python concepts. Depending on your pace and how much you practice the concepts, it could take a month or more.

2. **"Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin**  
   - **Estimated Time**: 2-3 weeks  
   - This book is more concise and can be read relatively quickly, especially if you focus on the specific ways that resonate with your current projects.

3. **"Python Cookbook" by David Beazley and Brian K. Jones**  
   - **Estimated Time**: 3-4 weeks  
   - This book is practical and hands-on, so you might want to spend time implementing the recipes, which could extend the t

In [40]:
chain_time1.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            

## Runnable Lambda

In [41]:
find_sum = lambda x: sum(x)

In [42]:
find_sum([1,2,5])

8

In [43]:
find_square = lambda x: x**2

In [44]:
find_square(5)

25

In [45]:
from langchain_core.runnables import RunnableLambda

In [46]:
runnable_sum = RunnableLambda(lambda x: sum(x))

In [47]:
runnable_sum.invoke([1,2,5])

8

In [48]:
runnable_square = RunnableLambda(lambda x: x**2)

In [49]:
runnable_square.invoke(10)

100

In [50]:
chain = runnable_sum | runnable_square

In [51]:
chain.invoke([1,2,11])

196

## The @chain Decorator

In [52]:
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables import chain

In [53]:
def find_sum(x):
    return sum(x)

def find_square(x):
    return x**2

In [54]:
chain1 = RunnableLambda(find_sum) | RunnableLambda(find_square)

In [55]:
chain1.invoke([2,8,9])

361

In [56]:
@chain
def runnable_sum(x):
    return sum(x)

@chain
def runnable_square(x):
    return x**2

In [57]:
chain2 = runnable_sum | runnable_square

In [58]:
chain2.invoke([3,9])

144

## Adding memory to chain

In [59]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [60]:
from langchain.memory import ConversationSummaryMemory
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [61]:
chat = ChatOpenAI(model="gpt-4o-mini", seed=365, temperature=0, max_tokens=100)

In [62]:
TEMPLATE = '''
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{message_log}

Human: 
{question}

AI:
'''

prompt_template = PromptTemplate.from_template(template=TEMPLATE)


In [63]:
chain = prompt_template | chat | StrOutputParser()

In [64]:
chain.invoke({'message_log': '', 'question': 'Can you give me an interesting fact?'})

"Sure! Did you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible! Honey's long shelf life is due to its low moisture content and acidic pH, which create an inhospitable environment for bacteria and microorganisms. So, if you have a jar of honey sitting in your pantry, it could last indefinitely!"

In [65]:
chat_memory = ConversationSummaryMemory(llm=ChatOpenAI(), memory_key='message_log')

C:\Users\matsv\AppData\Local\Temp\ipykernel_11816\1471810218.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  chat_memory = ConversationSummaryMemory(llm=ChatOpenAI(), memory_key='message_log')


In [66]:
chat_memory.load_memory_variables({})

{'message_log': ''}

In [67]:
RunnablePassthrough(message_log=chat_memory.load_memory_variables).invoke({'question': "Can you give me an interesting fact?"})

{'question': 'Can you give me an interesting fact?'}

In [68]:
RunnablePassthrough.assign(first_letter = lambda x: list(x['input'])[0]).invoke({'input': "fact?"})

{'input': 'fact?', 'first_letter': 'f'}

## Itemgetter

In [69]:
from operator import itemgetter

In [70]:
itemgetter(0)('hi')

'h'

In [71]:
itemgetter(2)([0,1,2,3,4])

2

In [73]:
itemgetter('message_log')({'message_log': 'hi'})

'hi'

In [75]:
RunnableLambda(itemgetter('message_log')).invoke({'message_log': 'hello'})

'hello'

In [78]:
RunnablePassthrough.assign(message_log=
                    RunnableLambda(chat_memory.load_memory_variables) | RunnableLambda(itemgetter('message_log'))).invoke(
    {'question': "Can you give me an interesting fact?"})

{'question': 'Can you give me an interesting fact?', 'message_log': ''}

# Adding Memory To chain Part 2

In [88]:
# question = "Can you give me an interesting fact i didn't know about?"
question = "Can you elaborate a bit more on this?"

In [89]:
output = RunnablePassthrough.assign(
    message_log= RunnableLambda(chat_memory.load_memory_variables) |
                itemgetter('message_log')).invoke(
    {'question': question})

In [90]:
prompt_value_template = prompt_template.invoke(output)

In [91]:
ai_message_output = chat.invoke(prompt_value_template)

In [92]:
response = StrOutputParser().invoke(ai_message_output)

In [93]:
chat_memory.save_context(inputs={'input': question}, outputs={'output': response})

In [94]:
chat_memory.load_memory_variables({})

{'message_log': "The human asks for an interesting fact they didn't know about, and the AI shares that honey never spoils due to its low moisture content and acidic pH. Honey found in ancient Egyptian tombs over 3,000 years old is still edible. The AI elaborates on honey's chemical composition, including its low moisture content and natural acidity, which prevent bacteria growth. Bees collect nectar from flowers to create honey, making it a truly fascinating and long-lasting food source."}

In [95]:
chain1 = (
    RunnablePassthrough.assign(
        message_log = RunnableLambda(chat_memory.load_memory_variables) | 
        itemgetter('message_log')) 
    | prompt_template 
    | chat 
    | StrOutputParser()
)

question = "Can you elaborate a bit more on this fact?"

response = chain1.invoke({'question':question})

chat_memory.save_context(inputs = {'input':question}, 
                         outputs = {'output':response})

response

"Absolutely! Honey's remarkable longevity is primarily due to its unique chemical composition. When bees collect nectar from flowers, they bring it back to the hive and process it by adding enzymes, which break down the sugars in the nectar. This process not only transforms the nectar into honey but also reduces its moisture content significantly.\n\nThe low moisture content is crucial because most bacteria and microorganisms require a certain level of moisture to thrive. Honey typically has a moisture content of around 17-18%, which is too low for"